# Introduction to Data Preprocessing

We will investigate common practices of data preprocessing with a toy example.


In [1]:
import pandas as pd
import numpy as np 
import sklearn

In [2]:
df = pd.read_csv("../Datasets/titanic3.csv")
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.00,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.92,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.00,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.00,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [6]:
df.shape

(1309, 14)

In [7]:
df.info()   #age, fare, cabin, embarked, boat, body, home.dest clearly have missing data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB


In [8]:
df.isna().sum() #you can check it also in this way

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

We have to decide how to handle the dataset. A first consideration can be made on the usefulness of the presented columns.

In [9]:
#drop columns Name, Ticket, Cabin, Boat, Body, Home.dest
df = df.drop(["name","ticket","cabin","boat","body", "home.dest"], axis=1)
#df.drop(["...."], axis=1, inplace=True)
df.head()

,pclass,survived,sex,age,sibsp,parch,fare,embarked
0,1,1,female,29.00,0,0,211.3375,S
1,1,1,male,0.92,1,2,151.5500,S
2,1,0,female,2.00,1,2,151.5500,S
3,1,0,male,30.00,1,2,151.5500,S
4,1,0,female,25.00,1,2,151.5500,S


We still have to deal with missing data. We can choose either to remove the rows that present missing data (but we risk losing a lot of information) or impute the missing values.

If you want to remove the rows with missing values you can simply type `df = df.dropna()`. We will proceed with the imputation instead.

In [10]:
df['age'] = df['age'].fillna(df['age'].mean())  #impute the mean value of the column for the missing data
df['fare'] = df['fare'].fillna(df['fare'].mean())

Now we can preprocess the categorical data, as most models are not able to handle them explicitly we resort to dummy variables.

In [11]:
df = pd.concat([df, pd.get_dummies(df['pclass']), pd.get_dummies(df['sex']), pd.get_dummies(df['embarked'])], axis=1) #concatenate column-wise
df.drop(["pclass","sex","embarked"], axis=1, inplace=True) #remove original columns, keep only the dummy encoding

In [12]:
df.head()   #we can see the new columns were added

,survived,age,sibsp,parch,fare,1,2,3,female,male,C,Q,S
0,1,29.00,0,0,211.3375,1,0,0,1,0,0,0,1
1,1,0.92,1,2,151.5500,1,0,0,0,1,0,0,1
2,0,2.00,1,2,151.5500,1,0,0,1,0,0,0,1
3,0,30.00,1,2,151.5500,1,0,0,0,1,0,0,1
4,0,25.00,1,2,151.5500,1,0,0,1,0,0,0,1


Something else that can be done while preprocessing a dataset is the normalization of the numerical variables.

In [13]:
#we will treat 'age' and 'fare' 
#sibsp' and 'parch' represent the number of sibling/spouses or parents/children aboard the ship, and can be left untouched

#for example, we can choose to divide by the absolute value of the maximum in order to have features ranging in [0,1]
def absolute_maximum_scaler(series):
    return series/series.abs().max()

for col in ['age', 'fare']:
    df[col] = absolute_maximum_scaler(df[col])

In [14]:
df.head()

,survived,age,sibsp,parch,fare,1,2,3,female,male,C,Q,S
0,1,0.3625,0,0,0.412503,1,0,0,1,0,0,0,1
1,1,0.0115,1,2,0.295806,1,0,0,0,1,0,0,1
2,0,0.0250,1,2,0.295806,1,0,0,1,0,0,0,1
3,0,0.3750,1,2,0.295806,1,0,0,0,1,0,0,1
4,0,0.3125,1,2,0.295806,1,0,0,1,0,0,0,1


Lastly, you would usually separate the response variable from the covariates.

In [15]:
y = df['survived']
X = df.iloc[:,1:]

#or you could write
#y = df['survived']
#X = df.drop(['survived'], axis=1)